# Scaling RLlib: From Laptop to Cluster

**Prerequisites**: Complete [02_rllib_basics](../02_rllib_basics/01_ray_setup.ipynb) and [03_custom_environments](../03_custom_environments/01_gymnasium_envs.ipynb)

RLlib is built on Ray, which means you can scale from 1 CPU to thousands with minimal code changes!

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                        WHY DISTRIBUTED RL?                                  │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  RL needs MILLIONS of environment interactions.                             │
│                                                                             │
│  1 environment:                     16 parallel environments:               │
│  ──────────────                     ─────────────────────────               │
│                                                                             │
│  ┌───────────┐                      ┌───┐ ┌───┐ ┌───┐ ┌───┐                │
│  │           │                      │Env│ │Env│ │Env│ │Env│                │
│  │    Env    │                      └─┬─┘ └─┬─┘ └─┬─┘ └─┬─┘                │
│  │           │                      ┌───┐ ┌───┐ ┌───┐ ┌───┐                │
│  └─────┬─────┘                      │Env│ │Env│ │Env│ │Env│                │
│        │                            └─┬─┘ └─┬─┘ └─┬─┘ └─┬─┘                │
│        v                            ┌───┐ ┌───┐ ┌───┐ ┌───┐                │
│   1000 steps/sec                    │Env│ │Env│ │Env│ │Env│                │
│                                     └─┬─┘ └─┬─┘ └─┬─┘ └─┬─┘                │
│   1M steps = 17 minutes             ┌───┐ ┌───┐ ┌───┐ ┌───┐                │
│                                     │Env│ │Env│ │Env│ │Env│                │
│                                     └───┘ └───┘ └───┘ └───┘                │
│                                           │                                │
│                                           v                                │
│                                     16,000 steps/sec                       │
│                                                                             │
│                                     1M steps = 1 minute!                    │
│                                                                             │
│  Training that took hours --> takes minutes with parallelization!           │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

## RLlib's Distributed Architecture

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    RLLIB DISTRIBUTED ARCHITECTURE                           │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│                      ┌────────────────────────────┐                         │
│                      │    ALGORITHM (Driver)      │                         │
│                      │                            │                         │
│                      │  - Coordinates everything  │                         │
│                      │  - Manages training loop   │                         │
│                      │  - Stores final policy     │                         │
│                      └─────────────┬──────────────┘                         │
│                                    │                                        │
│              ┌─────────────────────┼─────────────────────┐                  │
│              │                     │                     │                  │
│              v                     v                     v                  │
│     ┌─────────────────┐   ┌─────────────────┐   ┌─────────────────┐        │
│     │    LEARNER      │   │  REPLAY BUFFER  │   │   EVALUATION    │        │
│     │                 │   │  (off-policy)   │   │   WORKERS       │        │
│     │  - GPU training │   │                 │   │                 │        │
│     │  - Gradients    │   │  - Stores exp   │   │  - Test policy  │        │
│     │  - Updates      │   │  - Samples      │   │  - No training  │        │
│     └─────────────────┘   └─────────────────┘   └─────────────────┘        │
│                                                                             │
│     ────────────────────────────────────────────────────────────────        │
│                          ENVRUNNERS (Workers)                               │
│     ────────────────────────────────────────────────────────────────        │
│                                                                             │
│     ┌──────────────┐ ┌──────────────┐ ┌──────────────┐ ┌──────────────┐    │
│     │ EnvRunner 0  │ │ EnvRunner 1  │ │ EnvRunner 2  │ │ EnvRunner 3  │    │
│     │              │ │              │ │              │ │              │    │
│     │  ┌────────┐  │ │  ┌────────┐  │ │  ┌────────┐  │ │  ┌────────┐  │    │
│     │  │  Env   │  │ │  │  Env   │  │ │  │  Env   │  │ │  │  Env   │  │    │
│     │  └────────┘  │ │  └────────┘  │ │  └────────┘  │ │  └────────┘  │    │
│     │  ┌────────┐  │ │  ┌────────┐  │ │  ┌────────┐  │ │  ┌────────┐  │    │
│     │  │ Policy │  │ │  │ Policy │  │ │  │ Policy │  │ │  │ Policy │  │    │
│     │  │ (copy) │  │ │  │ (copy) │  │ │  │ (copy) │  │ │  │ (copy) │  │    │
│     │  └────────┘  │ │  └────────┘  │ │  └────────┘  │ │  └────────┘  │    │
│     └──────────────┘ └──────────────┘ └──────────────┘ └──────────────┘    │
│            │               │               │               │               │
│            └───────────────┴───────────────┴───────────────┘               │
│                                    │                                        │
│                      Experience (s, a, r, s') flows UP                      │
│                      Policy weights flow DOWN                               │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

**Key insight**: EnvRunners are Ray Actors! Each runs independently, collecting experience in parallel.

In [ ]:
# Suppress warnings
import warnings
import logging
warnings.filterwarnings("ignore")
logging.getLogger("ray").setLevel(logging.ERROR)

import ray
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.impala import IMPALAConfig
import numpy as np
import time

# Initialize Ray
ray.init(
    num_cpus=4,
    object_store_memory=1 * 1024 * 1024 * 1024,
    ignore_reinit_error=True,
)

print(f"Available resources: {ray.cluster_resources()}")

---

## Scaling with Workers

The simplest way to scale: add more EnvRunners!

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         WORKER SCALING                                      │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  num_env_runners controls parallelism:                                     │
│                                                                             │
│  num_env_runners=1        num_env_runners=4        num_env_runners=16     │
│  ─────────────────        ─────────────────        ──────────────────     │
│                                                                             │
│       ┌───┐                ┌───┐ ┌───┐             ┌───┐ ┌───┐ ┌───┐ ┌───┐│
│       │ E │                │ E │ │ E │             │ E │ │ E │ │ E │ │ E ││
│       └───┘                ├───┤ ├───┤             ├───┤ ├───┤ ├───┤ ├───┤│
│                            │ E │ │ E │             │ E │ │ E │ │ E │ │ E ││
│                            └───┘ └───┘             ├───┤ ├───┤ ├───┤ ├───┤│
│                                                    │ E │ │ E │ │ E │ │ E ││
│                                                    ├───┤ ├───┤ ├───┤ ├───┤│
│                                                    │ E │ │ E │ │ E │ │ E ││
│                                                    └───┘ └───┘ └───┘ └───┘│
│                                                                             │
│  1x throughput              4x throughput          16x throughput          │
│                                                                             │
│  TRADE-OFF:                                                                │
│  More workers = More throughput                                            │
│  More workers = More CPU usage                                             │
│  More workers = More memory usage                                          │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
def benchmark_workers(num_workers_list, n_iters=3):
    """Benchmark training speed with different worker counts."""
    results = {}
    
    for num_workers in num_workers_list:
        config = (
            PPOConfig()
            .environment("CartPole-v1")
            .framework("torch")
            .env_runners(
                num_env_runners=num_workers,
                num_envs_per_env_runner=1,
            )
            .training(
                train_batch_size=2000,
                sgd_minibatch_size=128,
            )
        )
        
        algo = config.build_algo()
        
        # Warm up
        algo.train()
        
        # Benchmark
        start = time.time()
        for _ in range(n_iters):
            algo.train()
        elapsed = time.time() - start
        
        samples_per_sec = (n_iters * 2000) / elapsed
        results[num_workers] = samples_per_sec
        
        print(f"Workers: {num_workers}, Samples/sec: {samples_per_sec:.0f}")
        algo.stop()
    
    return results

print("Benchmarking worker scaling...")
print("=" * 50)
results = benchmark_workers([1, 2], n_iters=3)

---

## Vectorized Environments

Even more parallelism: multiple envs PER worker!

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                      VECTORIZED ENVIRONMENTS                                │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  num_envs_per_env_runner=1          num_envs_per_env_runner=5              │
│  ────────────────────────────        ─────────────────────────────         │
│                                                                             │
│  ┌──────────────────────┐            ┌──────────────────────┐              │
│  │     EnvRunner        │            │     EnvRunner        │              │
│  │                      │            │                      │              │
│  │  ┌────────────────┐  │            │  ┌───┐ ┌───┐ ┌───┐   │              │
│  │  │      Env       │  │            │  │Env│ │Env│ │Env│   │              │
│  │  └────────────────┘  │            │  └───┘ └───┘ └───┘   │              │
│  │                      │            │  ┌───┐ ┌───┐         │              │
│  └──────────────────────┘            │  │Env│ │Env│         │              │
│                                      │  └───┘ └───┘         │              │
│  1 env per worker                    └──────────────────────┘              │
│                                                                             │
│                                      5 envs per worker                     │
│                                      (vectorized step!)                    │
│                                                                             │
│  WHY VECTORIZE?                                                            │
│  - Batch inference: compute actions for 5 envs at once                    │
│  - Better GPU utilization                                                  │
│  - Less overhead than separate workers                                     │
│                                                                             │
│  FORMULA:                                                                  │
│  total_envs = num_env_runners × num_envs_per_env_runner                   │
│                                                                             │
│  Example: 4 workers × 5 envs = 20 parallel environments!                  │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# High-throughput configuration
high_throughput_config = (
    PPOConfig()
    .environment("CartPole-v1")
    .framework("torch")
    
    .env_runners(
        num_env_runners=2,            # 2 worker processes
        num_envs_per_env_runner=4,    # 4 envs per worker = 8 total!
        rollout_fragment_length=200,  # Steps per collection
    )
    
    .training(
        train_batch_size=4000,        # 2 × 4 × 200 = 1600 → will wait for 3 collections
        sgd_minibatch_size=256,
        num_sgd_iter=10,
        lr=3e-4,
    )
)

print("High-throughput config:")
print(f"  Workers: {high_throughput_config.num_env_runners}")
print(f"  Envs per worker: {high_throughput_config.num_envs_per_env_runner}")
print(f"  Total parallel envs: {high_throughput_config.num_env_runners * high_throughput_config.num_envs_per_env_runner}")

---

## GPU Configuration

GPUs accelerate the LEARNING part (gradient computation).

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                          GPU USAGE IN RLLIB                                 │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  WHERE DOES GPU HELP?                                                       │
│  ────────────────────                                                       │
│                                                                             │
│  Data Collection (EnvRunners)          Learning (Trainer/Learner)           │
│  ────────────────────────────          ──────────────────────────           │
│                                                                             │
│  env.step() is Python                  Neural network forward/backward      │
│  --> CPU bound!                        --> GPU accelerated!                 │
│                                                                             │
│  GPU DOESN'T HELP HERE                 GPU HELPS HERE                       │
│                                                                             │
│  ────────────────────────────────────────────────────────────────────       │
│                                                                             │
│  CONFIGURATION:                                                             │
│                                                                             │
│  # Single GPU training                                                      │
│  .resources(num_gpus=1)                                                     │
│                                                                             │
│  # Multi-GPU with new Learner API                                           │
│  .learners(                                                                 │
│      num_learners=2,        # 2 learner workers                             │
│      num_gpus_per_learner=1 # Each gets 1 GPU                               │
│  )                                                                          │
│                                                                             │
│  ────────────────────────────────────────────────────────────────────       │
│                                                                             │
│  TYPICAL SETUP:                                                             │
│                                                                             │
│      ┌────────────────────────────────────────────────────┐                 │
│      │   Many CPUs for data collection (EnvRunners)       │                 │
│      │   ┌───┐ ┌───┐ ┌───┐ ┌───┐ ┌───┐ ┌───┐ ┌───┐ ┌───┐ │                 │
│      │   │CPU│ │CPU│ │CPU│ │CPU│ │CPU│ │CPU│ │CPU│ │CPU│ │                 │
│      │   └───┘ └───┘ └───┘ └───┘ └───┘ └───┘ └───┘ └───┘ │                 │
│      └────────────────────────┬───────────────────────────┘                 │
│                               │ experience                                  │
│                               v                                             │
│      ┌────────────────────────────────────────────────────┐                 │
│      │   1-2 GPUs for training (Learner)                  │                 │
│      │                 ┌─────┐ ┌─────┐                    │                 │
│      │                 │ GPU │ │ GPU │                    │                 │
│      │                 └─────┘ └─────┘                    │                 │
│      └────────────────────────────────────────────────────┘                 │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Single GPU configuration
single_gpu_config = (
    PPOConfig()
    .environment("CartPole-v1")
    .framework("torch")
    .env_runners(
        num_env_runners=4,
        num_cpus_per_env_runner=1,
    )
    .resources(
        num_gpus=1,  # GPU for training
        num_cpus_for_main_process=1,
    )
    .training(
        train_batch_size=4000,
        model={"fcnet_hiddens": [256, 256]},
    )
)

print("Single GPU config: num_gpus=1")

In [ ]:
# Multi-GPU with Learner API (RLlib 2.x+)
multi_gpu_config = (
    PPOConfig()
    .environment("CartPole-v1")
    .framework("torch")
    .env_runners(
        num_env_runners=8,
    )
    .learners(
        num_learners=2,           # 2 learner workers
        num_gpus_per_learner=1,   # Each gets 1 GPU
    )
    .training(
        train_batch_size=8000,    # Larger batch for multi-GPU
        minibatch_size=256,
    )
)

print("Multi-GPU config:")
print("  2 learners × 1 GPU each = 2 GPUs for training")
print("  8 EnvRunners for data collection")

---

## IMPALA: Built for Scale

For truly massive scale (100+ workers), use algorithms designed for it.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                              IMPALA                                         │
│               Importance Weighted Actor-Learner Architecture               │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  KEY IDEA: Decouple acting from learning                                   │
│                                                                             │
│  PPO (synchronous):              IMPALA (asynchronous):                    │
│  ─────────────────               ──────────────────────                    │
│                                                                             │
│  Collect ──> Train ──> Collect   Actors collect CONTINUOUSLY               │
│     │          ↑         │       Learner trains CONTINUOUSLY               │
│     └──── wait ┘         │                                                  │
│                                  ┌─────────────────────────┐               │
│  Workers wait while              │      LEARNER (GPU)      │               │
│  trainer updates                 │   ┌─────────────────┐   │               │
│                                  │   │  Train on any   │   │               │
│                                  │   │  available data │   │               │
│                                  │   └─────────────────┘   │               │
│                                  └──────────┬──────────────┘               │
│                                             │                              │
│                          ┌──────────────────┼──────────────────┐           │
│                          │                  │                  │           │
│                     ┌────┴────┐        ┌────┴────┐        ┌────┴────┐      │
│                     │ Actor 1 │        │ Actor 2 │        │Actor 100│      │
│                     │ (keeps  │        │ (keeps  │   ...  │ (keeps  │      │
│                     │ running)│        │ running)│        │ running)│      │
│                     └─────────┘        └─────────┘        └─────────┘      │
│                                                                             │
│  PROBLEM: Actors have OLD policy (not updated yet)                        │
│  SOLUTION: V-trace corrects for policy lag!                               │
│                                                                             │
│  USE IMPALA WHEN:                                                          │
│  • You have 50+ workers                                                    │
│  • Training is the bottleneck                                              │
│  • You want maximum throughput                                             │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# IMPALA configuration
impala_config = (
    IMPALAConfig()
    .environment("CartPole-v1")
    .framework("torch")
    .env_runners(
        num_env_runners=4,
        num_envs_per_env_runner=5,  # Vectorized
    )
    .resources(
        num_gpus=0,  # Would use 1+ in production
    )
    .training(
        lr=5e-4,
        train_batch_size=500,
        # V-trace parameters (IMPALA's secret sauce)
        vtrace=True,
        vtrace_clip_rho_threshold=1.0,
        vtrace_clip_pg_rho_threshold=1.0,
    )
)

print("IMPALA config created")
print("  V-trace enabled for off-policy correction")
print("  Total envs: 4 workers × 5 envs = 20")

---

## Ray Cluster: Multi-Node Training

For really big jobs, spread across multiple machines.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           RAY CLUSTER                                       │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  ┌─────────────────────────────────────────────────────────────────────┐   │
│  │                         HEAD NODE                                   │   │
│  │                                                                     │   │
│  │   ┌─────────────┐  ┌─────────────┐  ┌─────────────┐               │   │
│  │   │   Driver    │  │   GCS       │  │  Dashboard  │               │   │
│  │   │  (your code)│  │ (metadata)  │  │ (localhost: │               │   │
│  │   │             │  │             │  │     8265)   │               │   │
│  │   └─────────────┘  └─────────────┘  └─────────────┘               │   │
│  │                                                                     │   │
│  └─────────────────────────────────────────────────────────────────────┘   │
│                                   │                                        │
│                    ┌──────────────┼──────────────┐                        │
│                    │              │              │                        │
│                    v              v              v                        │
│  ┌──────────────────┐ ┌──────────────────┐ ┌──────────────────┐          │
│  │   WORKER NODE 1  │ │   WORKER NODE 2  │ │   WORKER NODE 3  │          │
│  │                  │ │                  │ │                  │          │
│  │  ┌────┐ ┌────┐   │ │  ┌────┐ ┌────┐   │ │  ┌────┐ ┌────┐   │          │
│  │  │ER 1│ │ER 2│   │ │  │ER 3│ │ER 4│   │ │  │ER 5│ │ER 6│   │          │
│  │  └────┘ └────┘   │ │  └────┘ └────┘   │ │  └────┘ └────┘   │          │
│  │                  │ │                  │ │                  │          │
│  │  16 CPUs, 64GB   │ │  16 CPUs, 64GB   │ │  16 CPUs, 64GB   │          │
│  └──────────────────┘ └──────────────────┘ └──────────────────┘          │
│                                                                            │
│  SETUP OPTIONS:                                                           │
│  ─────────────                                                            │
│  1. Manual: ray start --head / ray start --address=HEAD_IP                │
│  2. Cloud:  ray up cluster.yaml (AWS, GCP, Azure)                         │
│  3. K8s:    KubeRay operator                                              │
│                                                                            │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Example cluster config (for reference)
cluster_yaml = """
# Save as cluster.yaml
cluster_name: rllib-cluster

max_workers: 4

provider:
    type: aws
    region: us-west-2

head_node:
    InstanceType: m5.2xlarge  # 8 CPUs

worker_nodes:
    InstanceType: m5.2xlarge  # 8 CPUs each

setup_commands:
    - pip install "ray[rllib]" torch
"""

print("Cluster setup commands:")
print("  ray up cluster.yaml       # Start cluster")
print("  ray submit cluster.yaml train.py  # Run job")
print("  ray down cluster.yaml     # Stop cluster")

---

## Summary: Scaling Cheat Sheet

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                        SCALING CHEAT SHEET                                  │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  LAPTOP (4-8 CPUs, no GPU)                                                 │
│  ─────────────────────────                                                 │
│  .env_runners(num_env_runners=2, num_envs_per_env_runner=2)               │
│  .resources(num_gpus=0)                                                    │
│                                                                             │
│  WORKSTATION (16+ CPUs, 1 GPU)                                             │
│  ──────────────────────────────                                            │
│  .env_runners(num_env_runners=8, num_envs_per_env_runner=4)               │
│  .resources(num_gpus=1)                                                    │
│                                                                             │
│  SMALL CLUSTER (4 nodes, 64 CPUs, 4 GPUs)                                  │
│  ─────────────────────────────────────────                                 │
│  .env_runners(num_env_runners=32, num_envs_per_env_runner=4)              │
│  .learners(num_learners=4, num_gpus_per_learner=1)                        │
│                                                                             │
│  LARGE CLUSTER (use IMPALA)                                                │
│  ───────────────────────────                                               │
│  IMPALAConfig()                                                            │
│  .env_runners(num_env_runners=100, num_envs_per_env_runner=5)             │
│  .resources(num_gpus=8)                                                    │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

## Key Takeaways

1. **Scale workers** (`num_env_runners`) for more data collection throughput

2. **Vectorize** (`num_envs_per_env_runner`) for more efficient batching

3. **Use GPU** for training acceleration (the Learner)

4. **IMPALA** for massive scale (100+ workers)

5. **Ray Cluster** for multi-node training

## What's Next?

```
┌──────────────────┐          ┌──────────────────┐          ┌──────────────────┐
│  05 Distributed  │          │    06 Ray Tune   │          │  07 Production   │
│  (you are here)  │   ───>   │                  │   ───>   │                  │
│                  │          │  Find optimal    │          │  Deploy your     │
│  - Workers       │          │  hyperparameters │          │  trained policy  │
│  - GPUs          │          │                  │          │                  │
│  - Clusters      │          │                  │          │                  │
└──────────────────┘          └──────────────────┘          └──────────────────┘
```

In [ ]:
ray.shutdown()